In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline, AutoModelForMaskedLM, AutoTokenizer
import re
from tqdm import tqdm
from itertools import permutations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_candidates_setwise(Ci,perms, wk):
    Ci_final = set()
    for cj in Ci:
        # Ci.remove(cj)
        c0j = cj.split()
        for tup in perms:
            print(tup)
            for i in range(len(wk)):
                c0j[tup[i]]= wk[i]
        # print(Ak)
        # for a in Ak:
        #     c0j = cj.split()
        #     c0j[a[1]] = a[0]

            print(c0j)
            Ci_final.add(' '.join((c0j)))
            c0j =  cj.split()
    return Ci_final

def candidate_set_generation(xi, pi0, Ti, T0i):
    c0 = ' '.join(['<mask>'] * len(pi0))
    Ci = {c0}
    Tshared = Ti.intersection(T0i)
    print(Tshared)
    for tk in Tshared:
        print(tk)
        Wk = [word for word, pos in xi if pos == tk]
        Sk = [i for i, pos in enumerate(pi0) if pos[1] == tk]
        Ak = []
        print(f'Wk = {Wk} Sk = {Sk} ')
        if len(Wk)==1 and len(Sk)>1:
            for i in Sk:
                Ak.append((Wk[0],i))
        elif len(Sk)==1: 
            for word in Wk:
                Ak.append((word,Sk[0]))
        elif len(Sk)>len(Wk):
            perms = list(permutations(Sk,len(Wk)))
        # else:
        print(len(perms))
            # //for perm in perms:
            # for word in Wk:
            #     for i in Sk:
            #         Ak.append((word,i))
        # print('f Ak = ',Ak)
        # print(len(Ak))
        Ci_tag = get_candidates_setwise(Ci,perms,Wk)
        Ci = Ci.union(Ci_tag)
    return Ci

In [ ]:
# offensive sentence 
xi = 'here comes the beginning of another fucking recession'
# offensive sentence pos sequence 
tag_xi = ['RB', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'BW', 'NN']
# non-offensive sentence
pi0 = 'this was predicted years ago on the superstonk sub once they start closing that whole recession will kick in and the market will bleed greedy money stealing criminals have bled the citizens dry and still want more'
pi0 = pi0.split(' ')
# non-offensive sentence pos tags 
tag_pi0 = ['DT', 'VBD', 'VBN', 'NNS', 'RB', 'IN', 'DT', 'NN', 'NN', 'IN', 'PRP', 'VBP', 'VBG', 'DT', 'JJ', 'NN', 'MD', 'VB', 'RP', 'CC', 'DT', 'NN', 'MD', 'VB', 'JJ', 'NN', 'VBG', 'NNS', 'VBP', 'VBN', 'DT', 'NNS', 'JJ', 'CC', 'RB', 'VBP', 'JJR']
tag_pi0 = tag_pi0
# unique pos tags/tokens of non-offensive sentence
T0i = set(tag_pi0)
# unique pos tags/tokens of offensive sentence
Ti = set(tag_xi)
# tuple set of word in offensive sentence and its corresponding pos tag
xi_mod = [(word,tag) for word,tag in zip(xi.split(),tag_xi)]
# tuple set of word in non-offensive sentence and its corresponding pos tag
pi0_mod = [(word,tag) for word,tag in zip(pi0,tag_pi0)]
candidate_sentences = candidate_set_generation(xi=xi_mod,pi0=pi0_mod,Ti=Ti,T0i=T0i)

In [ ]:
# Load the pre-trained DistilRoBERTa model and tokenizer
model_name = "distilroberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Initialize the pipeline for masked language modeling
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)


TODO: Fine tuned model

In [ ]:
def get_mask_filled_candidate_sentences(candidate_sentences):
    mask_filled_candidate_sentences=[]
    for i,sentence in enumerate(tqdm(candidate_sentences)):
        number_of_masked_tokens = sentence.count('<mask>')
        for i in range(number_of_masked_tokens):
            if i==number_of_masked_tokens-1:
                sentence=fill_mask(sentence.replace("<mask>", tokenizer.mask_token), top_k=1)[0]['sequence']
            else:
                sentence=fill_mask(sentence.replace("<mask>", tokenizer.mask_token), top_k=1)[0][0]['sequence']
            sentence = re.sub(r'<s>|</s>', ' ',sentence)
        mask_filled_candidate_sentences.append(sentence)
    return mask_filled_candidate_sentences

In [ ]:
candidate_sentences = list(candidate_sentences)
mask_filled_candidate_sentences = get_mask_filled_candidate_sentences(candidate_sentences)

In [ ]:
# mask_filled_candidate_sentences

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def get_highest_bleu_sentence(candidate_sentences, reference_sentence):
    max_bleu_score = 0
    max_bleu_sentence = None
    
    # Calculate BLEU score for each candidate sentence
    for sentence in candidate_sentences:
        bleu_score = sentence_bleu([reference_sentence], sentence.split())
        
        # Update max BLEU score and corresponding sentence
        if bleu_score > max_bleu_score:
            max_bleu_score = bleu_score
            max_bleu_sentence = sentence
    
    return max_bleu_sentence, max_bleu_score

print(xi)
get_highest_bleu_sentence(mask_filled_candidate_sentences, xi)


In [ ]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer

# Load the pre-trained DistilRoBERTa model and tokenizer
model_name = "distilroberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the sentence with multiple masks
sentence = "The <mask> brown <mask> jumps over the lazy <mask>."

# Initialize the pipeline for masked language modeling
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

# Use the pipeline to fill in the first masked token
results_1 = fill_mask(sentence.replace("[MASK]", tokenizer.mask_token), top_k=5)
sentence = results_1[0][0]['sequence']
print(sentence)

# Use the pipeline to fill in the second masked token
results_2 = fill_mask(sentence.replace("[MASK]", tokenizer.mask_token), top_k=5)
sentence = results_2[0][0]['sequence']
print(sentence)
# Use the pipeline to fill in the third masked token
results_3 = fill_mask(sentence.replace("[MASK]", tokenizer.mask_token), top_k=5)
sentence = results_3[0]['sequence']
print(sentence)
# Print the top predictions for each mask
# for result in results_1:
    # print(result['sequence'].replace(tokenizer.mask_token, "quick"))

# for result in results_2:
#     print(result['sequence'].replace(tokenizer.mask_token, "fox"))


In [ ]:
Sk = [1,2,3,4,5]
Wk= ['a','b','c']
c = list(permutations(Sk,len(Wk)))
for i in c:
    print(i)